# Lab 3

In [1]:
import pandas as pd

from modules.tfidf import GeniusClient

In [2]:
client = GeniusClient()

In [3]:
artists = (
    'Slipknot',
    'Eminem',
    'Billie Eilish',
    'Radiohead',
    'Jack Black',
    'Kendrick Lamar',
    'Ed Sheeran',
    'Miley Cyrus',
    'The Killers',
    'Alex G',
    'Katy Perry',
    'OneRepublic',
    'Justin Bieber',
    'Bryan Adams',
    'Machine Head',
    'Architects',
    'Cage The Elephant',
    'The White Stripes',
    'Pierce The Veil',
    'Scorpions'
)

In [4]:
len(artists)

20

In [6]:
lyric_links = []

In [9]:
for artist in artists:
    lyric_links.extend(client.song_list(artist))

In [10]:
len(lyric_links)

200

In [63]:
def find_lyric(page: str) -> str:
    for line in page.text.split('\n'):
        if line.strip().startswith('window.__PRELOADED_STATE__ = JSON.parse(\''):
            paragraph = line.replace('window.__PRELOADED_STATE__ = JSON.parse(\'', '').replace("\\');", '').split('\\"body\\":{\\"html\\":\\"')[1].split('\\",\\"children\\":')[0].replace('\\\\n', '').replace("\\'", '').replace('\\/', '').replace('<br>', ' ')

            return BeautifulSoup(paragraph, 'html.parser').text

In [64]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def normalize_text(text):
    text = text.lower()
    words = text.split()

    words = list(filter(lambda x: x != '', words))
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmas)

In [65]:
import requests
from bs4 import BeautifulSoup
import re

def get_lyrics(link):
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; YourBot/0.1; +http://yourdomain.com/bot)"
    }
    page = requests.get(link, headers=headers)
    lyric = find_lyric(page)
    lyric = re.sub(r'\[.*?\]', '', lyric)  # Удаляем аннотации типа [Chorus]
    lyric = re.sub(r'\n+', ' ', lyric)  # Убираем лишние новые строки
    lyric = lyric.replace('-', ' ')  # Заменяем тире на пробел
    lyric = re.sub(r'[^\w\s]', '', lyric)  # Удаляем пунктуацию
    lyric = re.sub(r'(\w)([A-Z])', r'\1 \2', lyric)  # Если слово слилось с заглавной буквы
    lyric = re.sub(r'\s+', ' ', lyric)  # Убираем лишние пробелы
    lyric = lyric.lower()
    lyric = normalize_text(lyric)
    return lyric

In [69]:
norm_l = []

In [70]:
import time

for lyric_link in lyric_links:
    normalized_lyric = get_lyrics(lyric_link)
    time.sleep(1)
    norm_l.append(normalized_lyric)

In [82]:
import pandas as pd
pd.DataFrame(norm_l).to_csv('./norm_l.csv')
pd.DataFrame(lyric_links).to_csv('./lyric_links.csv')
pd.DataFrame(artists).to_csv('./artists.csv')

In [73]:
len(norm_l)

200

In [107]:
import math

def compute_tf(text):
    words = text.split()
    freq = Counter(words)
    tf = {word: count / len(words) for word, count in freq.items()}
    return tf


def compute_idf(documents):
    N = len(documents)
    word_doc_count = Counter()
    for doc in documents:
        words = set(doc.split())
        word_doc_count.update(words)
    idf = {word: math.log(N / count) for word, count in word_doc_count.items()}
    return idf


def compute_tf_idf(tf, idf):
    return {word: tf[word] * idf.get(word, 0) for word in tf}

In [108]:


[doc.split() for doc in norm_l[20*0:20]]

[['bury',
  'all',
  'your',
  'secrets',
  'in',
  'my',
  'skin',
  'come',
  'away',
  'with',
  'innocence',
  'and',
  'leave',
  'me',
  'with',
  'my',
  'sins',
  'the',
  'air',
  'around',
  'me',
  'still',
  'feels',
  'like',
  'a',
  'cage',
  'and',
  'love',
  'is',
  'just',
  'a',
  'camouflage',
  'for',
  'what',
  'resembles',
  'rage',
  'again',
  'so',
  'if',
  'you',
  'love',
  'me',
  'let',
  'me',
  'go',
  'and',
  'run',
  'away',
  'before',
  'i',
  'know',
  'my',
  'heart',
  'is',
  'just',
  'too',
  'dark',
  'to',
  'care',
  'i',
  'cant',
  'destroy',
  'what',
  'isnt',
  'there',
  'deliver',
  'me',
  'into',
  'my',
  'fate',
  'if',
  'im',
  'alone',
  'i',
  'cannot',
  'hate',
  'i',
  'dont',
  'deserve',
  'to',
  'have',
  'you',
  'ooh',
  'my',
  'smile',
  'was',
  'taken',
  'long',
  'ago',
  'if',
  'i',
  'can',
  'change',
  'i',
  'hope',
  'i',
  'never',
  'know',
  'i',
  'still',
  'press',
  'your',
  'letters',
  'to',

In [109]:
len(norm_l[38])

567

In [110]:
from collections import Counter

for idx, artist in enumerate(artists):
    start = 10 * idx
    end = 10 * (idx + 1)
    print(f"Processing artist {artist} with slice {start}:{end}")
    a_ls = norm_l[start:end]

    print(f"Number of songs for {artist}: {len(a_ls)}")

    tokenized_docs = []
    for doc in a_ls:
        if isinstance(doc, str) and doc.strip():
            tokenized_docs.extend(doc.split())

    print(f"Number of tokens for {artist}: {len(tokenized_docs)}")

    word_counts = Counter(tokenized_docs)
    print(f'Artist: {artist}')
    print(f'Top words: {word_counts.most_common(10)}')
    print()

Processing artist Slipknot with slice 0:10
Number of songs for Slipknot: 10
Number of tokens for Slipknot: 3510
Artist: Slipknot
Top words: [('the', 163), ('you', 134), ('i', 127), ('me', 85), ('to', 67), ('in', 63), ('my', 63), ('it', 55), ('and', 53), ('a', 49)]

Processing artist Eminem with slice 10:20
Number of songs for Eminem: 10
Number of tokens for Eminem: 9935
Artist: Eminem
Top words: [('the', 326), ('you', 313), ('i', 272), ('a', 266), ('and', 229), ('to', 224), ('im', 179), ('it', 143), ('me', 135), ('in', 118)]

Processing artist Billie Eilish with slice 20:30
Number of songs for Billie Eilish: 10
Number of tokens for Billie Eilish: 2619
Artist: Billie Eilish
Top words: [('you', 170), ('i', 147), ('me', 62), ('to', 59), ('it', 54), ('the', 47), ('a', 46), ('like', 45), ('im', 37), ('my', 34)]

Processing artist Radiohead with slice 30:40
Number of songs for Radiohead: 10
Number of tokens for Radiohead: 1443
Artist: Radiohead
Top words: [('i', 61), ('the', 52), ('you', 49)

In [111]:
documents = norm_l
idf = compute_idf(documents)


for idx, song in enumerate(lyric_links):
    tf = compute_tf(norm_l[idx])
    tf_idf = compute_tf_idf(tf, idf)

    if tf_idf:
        max_word = max(tf_idf, key=tf_idf.get)
        max_score = tf_idf[max_word]
        print(f"Song: {song}")
        print(f"  Most significant word: '{max_word}' with TF-IDF score of {max_score:.4f}")
    else:
        print(f"Song: {song} has no valid words.")


print("\nTF-IDF по каждому артисту")
for idx, artist in enumerate(artists):
    start = idx * 10
    end = start + 10
    artist_songs = norm_l[start:end]

    if not artist_songs:
        print(f"Artist '{artist}' has no songs.")
        continue

    all_lyrics = " ".join(artist_songs)
    tf_all = compute_tf(all_lyrics)
    tf_idf_all = compute_tf_idf(tf_all, idf)

    if tf_idf_all:
        max_word = max(tf_idf_all, key=tf_idf_all.get)
        max_score = tf_idf_all[max_word]
        print(f"Artist: {artist}")
        print(f"  Most significant word: '{max_word}' with score {max_score:.4f}")
    else:
        print(f"Artist '{artist}' has no valid words.")


Song: https://genius.com/Slipknot-snuff-lyrics
  Most significant word: 'ago' with TF-IDF score of 0.0400
Song: https://genius.com/Slipknot-psychosocial-lyrics
  Most significant word: 'psychosocial' with TF-IDF score of 0.2990
Song: https://genius.com/Slipknot-duality-lyrics
  Most significant word: 'insane' with TF-IDF score of 0.0986
Song: https://genius.com/Slipknot-sic-lyrics
  Most significant word: 'sic' with TF-IDF score of 0.0902
Song: https://genius.com/Slipknot-unsainted-lyrics
  Most significant word: 'letting' with TF-IDF score of 0.0691
Song: https://genius.com/Slipknot-wait-and-bleed-lyrics
  Most significant word: 'kneel' with TF-IDF score of 0.0878
Song: https://genius.com/Slipknot-the-devil-in-i-lyrics
  Most significant word: 'devil' with TF-IDF score of 0.1738
Song: https://genius.com/Slipknot-eyeless-lyrics
  Most significant word: 'marlon' with TF-IDF score of 0.1428
Song: https://genius.com/Slipknot-people-shit-lyrics
  Most significant word: 'equal' with TF-IDF 